# CS598 Deep Learning for Healthcare Final Project
## Reproduction of Deepr: A Convolutional Net for Medical Records
### Juan Alvarez Martinez, Shane Sepac

In [2]:
### TODO: Include summary and report of findings here [200 words]

## Load MIMIC-III Dataset. 
- Ensure you have all environment variables from .env.sample copied and instantiated in a .env file!
- Ensure required dependencies are available to kernel

In [19]:
!pip3 install boto3 python-dotenv pandas

In [ ]:
%load ./utils.py

In [24]:
import pandas as pd
from utils import readS3CSV

df = readS3CSV("D_CPT.csv")
print(df)

     ROW_ID  CATEGORY SECTIONRANGE              SECTIONHEADER SUBSECTIONRANGE   
0         1         1  99201-99499  Evaluation and management     99201-99216  \
1         2         1  99201-99499  Evaluation and management     99217-99220   
2         3         1  99201-99499  Evaluation and management     99221-99239   
3         4         1  99201-99499  Evaluation and management     99241-99255   
4         5         1  99201-99499  Evaluation and management     99261-99263   
..      ...       ...          ...                        ...             ...   
129     130         2  0001F-7025F    Performance measurement     5005F-5100F   
130     131         2  0001F-7025F    Performance measurement     6005F-6045F   
131     132         2  0001F-7025F    Performance measurement     7010F-7025F   
132     133         3  0016T-0207T        Emerging technology     0016T-0207T   
133     134         3  0256T-0259T        Emerging technology     0256T-0259T   

                           